In [1]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import cv2
from pathlib import Path

In [7]:
# class VideoUploaderApp:
#     def __init__(self, master):
#         self.master = master
#         self.master.title("Video Uploader")

#         # Variables
#         self.video_path = tk.StringVar()

#         # Create Widgets
#         self.create_widgets()

#     def create_widgets(self):
#         # Video Path Entry
#         tk.Label(self.master, text="Video Path:").grid(row=0, column=0, pady=10)
#         tk.Entry(self.master, textvariable=self.video_path, width=40).grid(row=0, column=1, padx=10, pady=10)

#         # Browse Button
#         tk.Button(self.master, text="Browse", command=self.browse_video).grid(row=0, column=2, pady=10)

#         # Upload Button
#         tk.Button(self.master, text="Upload", command=self.upload_video).grid(row=1, column=1, pady=10)

#     def browse_video(self):
#         file_path = filedialog.askopenfilename(filetypes=[("Video Files", "*.mp4;*.avi")])
#         if file_path:
#             self.video_path.set(file_path)

# #     def upload_video(self):
# #         # Implement your upload logic here
# #         # You can use self.video_path.get() to get the selected video path
# #         print("Uploading video:", self.video_path.get())
        
#     def upload_video(self):
#         video_path = self.video_path.get()

#         if video_path:
#             # Implement your upload logic here
#             # For example, you can display a message in the GUI
#             success_message = f"Video '{video_path}' uploaded successfully!"
#             tk.Label(self.master, text=success_message, fg="green").grid(row=2, column=1, pady=10)
#         else:
#             # If no video path is selected, show an error message
#             error_message = "Please select a video before uploading."
#             tk.Label(self.master, text=error_message, fg="red").grid(row=2, column=1, pady=10)

In [8]:
# def main():
#     root = tk.Tk()
#     app = VideoUploaderApp(root)
#     root.mainloop()

# if __name__ == "__main__":
#     main()

In [7]:
#------------------ WITHOUT YOLO -------------------------- #
class VideoUploaderApp:
    def __init__(self, master):
        self.master = master
        self.master.title("Video Uploader")

        # Variables
        self.video_path = tk.StringVar()
        self.video_capture = None
        self.video_canvas = None
        self.start_x = None
        self.start_y = None
        self.bbox_coords = None
        self.paused = True

        # Create Widgets
        self.create_widgets()

    def create_widgets(self):
        # Video Path Entry
        tk.Label(self.master, text="Video Path:").grid(row=0, column=0, pady=10)
        tk.Entry(self.master, textvariable=self.video_path, width=40).grid(row=0, column=1, padx=10, pady=10)

        # Browse Button
        tk.Button(self.master, text="Browse", command=self.browse_video).grid(row=0, column=2, pady=10)

        # Upload Button
        tk.Button(self.master, text="Upload", command=self.upload_video).grid(row=1, column=1, pady=10)

        # Play/Pause/Replay Buttons
        tk.Button(self.master, text="Play", command=self.play_video).grid(row=1, column=0, pady=10)
        tk.Button(self.master, text="Pause", command=self.pause_video).grid(row=1, column=2, pady=10)
        tk.Button(self.master, text="Replay", command=self.replay_video).grid(row=1, column=3, pady=10)

        # Video Canvas
        self.video_canvas = tk.Canvas(self.master)
        self.video_canvas.grid(row=2, column=0, columnspan=4, pady=10)

    def browse_video(self):
        file_path = filedialog.askopenfilename(filetypes=[("Video Files", "*.mp4;*.avi")])
        if file_path:
            self.video_path.set(file_path)
            self.load_video()

    def load_video(self):
        video_path = self.video_path.get()
        if video_path:
            self.video_capture = cv2.VideoCapture(video_path)

            # Show the first frame
            ret, frame = self.video_capture.read()
            if ret:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                img = Image.fromarray(frame)
                img_tk = ImageTk.PhotoImage(img)

                self.video_canvas.config(width=img_tk.width(), height=img_tk.height())
                self.video_canvas.create_image(0, 0, anchor=tk.NW, image=img_tk)
                self.video_canvas.image = img_tk

                self.video_canvas.bind("<ButtonPress-1>", self.on_click)
                self.video_canvas.bind("<B1-Motion>", self.on_drag)
                self.video_canvas.bind("<ButtonRelease-1>", self.on_release)

                # If the video is paused, don't start playing
                if not self.paused:
                    self.show_frame()

    def show_frame(self):
        if not self.paused and self.video_capture.isOpened():
            ret, frame = self.video_capture.read()

            if ret:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                if self.bbox_coords:
                    # Draw bounding box on the frame
                    start_x, start_y, end_x, end_y = self.bbox_coords
                    cv2.rectangle(frame, (start_x, start_y), (end_x, end_y), (0, 255, 0), 2)

                img = Image.fromarray(frame)
                img_tk = ImageTk.PhotoImage(img)

                self.video_canvas.config(width=img_tk.width(), height=img_tk.height())
                self.video_canvas.create_image(0, 0, anchor=tk.NW, image=img_tk)
                self.video_canvas.image = img_tk

                self.video_canvas.bind("<ButtonPress-1>", self.on_click)
                self.video_canvas.bind("<B1-Motion>", self.on_drag)
                self.video_canvas.bind("<ButtonRelease-1>", self.on_release)

                self.master.after(10, self.show_frame)
            else:
                # If video is finished, release the capture
                self.video_capture.release()

    def on_click(self, event):
        self.start_x = event.x
        self.start_y = event.y

    def on_drag(self, event):
        self.video_canvas.delete("bbox")
        self.video_canvas.create_rectangle(
            self.start_x, self.start_y, event.x, event.y,
            outline="red", width=2, tags="bbox"
        )

    def on_release(self, event):
        # Save bounding box coordinates
        self.bbox_coords = (self.start_x, self.start_y, event.x, event.y)

    def play_video(self):
        self.paused = False
        self.show_frame()

    def pause_video(self):
        self.paused = True

    def replay_video(self):
        # Stop the video if it's currently playing
        self.pause_video()

        # Reset the bounding box coordinates
        self.bbox_coords = None

        # Reset the video capture
        self.video_capture.release()
        self.load_video()

        # Start playing the video if not paused
        if not self.paused:
            self.play_video()

    def upload_video(self):
        # Implement your upload logic here
        # You can use self.video_path.get() and self.bbox_coords to get the selected video path and bounding box coordinates
        print("Uploading video:", self.video_path.get())
        print("Bounding Box Coordinates:", self.bbox_coords)

In [8]:
# class VideoUploaderApp:
#     def __init__(self, master):
#         self.master = master
#         self.master.title("Video Uploader")

#         # Variables
#         self.video_path = tk.StringVar()
#         self.video_capture = None
#         self.video_canvas = None
#         self.start_x = None
#         self.start_y = None
#         self.bbox_coords = None
#         self.paused = True

# #         # YOLO
# #         self.yolo_net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
# #         self.yolo_classes = []
# #         with open("coco.names", "r") as f:
# #             self.yolo_classes = [line.strip() for line in f]
#         # YOLOv5
#         self.yolo_model = yolo_inference.YOLOv5Model('G:\FAST\Semester 7\Deep Learning\YOLO\yolov5/yolov5s.pt')

#         # Create Widgets
#         self.create_widgets()

#     def create_widgets(self):
#         # Video Path Entry
#         tk.Label(self.master, text="Video Path:").grid(row=0, column=0, pady=10)
#         tk.Entry(self.master, textvariable=self.video_path, width=40).grid(row=0, column=1, padx=10, pady=10)

#         # Browse Button
#         tk.Button(self.master, text="Browse", command=self.browse_video).grid(row=0, column=2, pady=10)

#         # Upload Button
#         tk.Button(self.master, text="Upload", command=self.upload_video).grid(row=1, column=1, pady=10)

#         # Play/Pause/Replay Buttons
#         tk.Button(self.master, text="Play", command=self.play_video).grid(row=1, column=0, pady=10)
#         tk.Button(self.master, text="Pause", command=self.pause_video).grid(row=1, column=2, pady=10)
#         tk.Button(self.master, text="Replay", command=self.replay_video).grid(row=1, column=3, pady=10)

#         # Video Canvas
#         self.video_canvas = tk.Canvas(self.master)
#         self.video_canvas.grid(row=2, column=0, columnspan=4, pady=10)
    
#     def detect_vehicles(self, frame):
#         if self.bbox_coords:
#             start_x, start_y, end_x, end_y = self.bbox_coords
#             roi = frame[start_y:end_y, start_x:end_x]

#             # YOLO object detection
#             blob = cv2.dnn.blobFromImage(roi, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
#             self.yolo_net.setInput(blob)
#             outs = self.yolo_net.forward(self.get_output_layers())

#             class_ids = []
#             confidences = []
#             boxes = []

#             for out in outs:
#                 for detection in out:
#                     scores = detection[5:]
#                     class_id = np.argmax(scores)
#                     confidence = scores[class_id]
#                     if confidence > 0.5 and class_id == 2:  # Class ID 2 corresponds to vehicles
#                         center_x, center_y, w, h = (detection[0:4] * np.array([roi.shape[1], roi.shape[0], roi.shape[1], roi.shape[0]])).astype('int')
#                         x = int(center_x - w / 2)
#                         y = int(center_y - h / 2)
#                         class_ids.append(class_id)
#                         confidences.append(float(confidence))
#                         boxes.append([x, y, w, h])

#             indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

#             for i in indices:
#                 i = i[0]
#                 box = boxes[i]
#                 x, y, w, h = box
#                 cv2.rectangle(roi, (x, y), (x + w, y + h), (0, 255, 0), 2)

#             return roi

#     def get_output_layers(self):
#         layer_names = self.yolo_net.getUnconnectedOutLayersNames()
#         return [layer_names[i[0] - 1] for i in self.yolo_net.getUnconnectedOutLayers()]

#     def browse_video(self):
#         file_path = filedialog.askopenfilename(filetypes=[("Video Files", "*.mp4;*.avi")])
#         if file_path:
#             self.video_path.set(file_path)
#             self.load_video()

#     def load_video(self):
#         video_path = self.video_path.get()
#         if video_path:
#             self.video_capture = cv2.VideoCapture(video_path)

#             # Show the first frame
#             ret, frame = self.video_capture.read()
#             if ret:
#                 frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#                 img = Image.fromarray(frame)
#                 img_tk = ImageTk.PhotoImage(img)

#                 self.video_canvas.config(width=img_tk.width(), height=img_tk.height())
#                 self.video_canvas.create_image(0, 0, anchor=tk.NW, image=img_tk)
#                 self.video_canvas.image = img_tk

#                 self.video_canvas.bind("<ButtonPress-1>", self.on_click)
#                 self.video_canvas.bind("<B1-Motion>", self.on_drag)
#                 self.video_canvas.bind("<ButtonRelease-1>", self.on_release)

#                 # If the video is paused, don't start playing
#                 if not self.paused:
#                     self.show_frame()

#     def show_frame(self):
#         if not self.paused and self.video_capture.isOpened():
#             ret, frame = self.video_capture.read()

#             if ret:
#                 frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

#                 if self.bbox_coords:
                    
#                     detected_roi = self.detect_vehicles(frame)
                    
#                     # Draw bounding box on the frame
#                     start_x, start_y, end_x, end_y = self.bbox_coords
#                     cv2.rectangle(frame, (start_x, start_y), (end_x, end_y), (0, 255, 0), 2)

#                 img = Image.fromarray(frame)
#                 img_tk = ImageTk.PhotoImage(img)

#                 self.video_canvas.config(width=img_tk.width(), height=img_tk.height())
#                 self.video_canvas.create_image(0, 0, anchor=tk.NW, image=img_tk)
#                 self.video_canvas.image = img_tk

#                 self.video_canvas.bind("<ButtonPress-1>", self.on_click)
#                 self.video_canvas.bind("<B1-Motion>", self.on_drag)
#                 self.video_canvas.bind("<ButtonRelease-1>", self.on_release)

#                 self.master.after(10, self.show_frame)
#             else:
#                 # If video is finished, release the capture
#                 self.video_capture.release()

#     def on_click(self, event):
#         self.start_x = event.x
#         self.start_y = event.y

#     def on_drag(self, event):
#         self.video_canvas.delete("bbox")
#         self.video_canvas.create_rectangle(
#             self.start_x, self.start_y, event.x, event.y,
#             outline="red", width=2, tags="bbox"
#         )

#     def on_release(self, event):
#         # Save bounding box coordinates
#         self.bbox_coords = (self.start_x, self.start_y, event.x, event.y)

#     def play_video(self):
#         self.paused = False
#         self.show_frame()

#     def pause_video(self):
#         self.paused = True

#     def replay_video(self):
#         # Stop the video if it's currently playing
#         self.pause_video()

#         # Reset the bounding box coordinates
#         self.bbox_coords = None

#         # Reset the video capture
#         self.video_capture.release()
#         self.load_video()

#         # Start playing the video if not paused
#         if not self.paused:
#             self.play_video()
        
# #     def upload_video(self):
# #         # Implement your upload logic here
# #         # You can use self.video_path.get() and self.bbox_coords to get the selected video path and bounding box coordinates
# #         print("Uploading video:", self.video_path.get())
# #         print("Bounding Box Coordinates:", self.bbox_coords)
    
#     def upload_video(self):
#         video_path = self.video_path.get()
#         bbox_coords = self.bbox_coords

#         if video_path and bbox_coords:
#             print("Uploading video:", video_path)
#             print("Bounding Box Coordinates:", bbox_coords)

#             # Perform YOLOv5 detection within the bounding box
#             self.detect_objects(video_path, bbox_coords)
    
#     def detect_objects(self, video_path, bbox_coords):
#         start_x, start_y, end_x, end_y = bbox_coords

#         # Open video
#         video_capture = cv2.VideoCapture(video_path)

#         # Process video frame by frame
#         while True:
#             ret, frame = video_capture.read()

#             if not ret:
#                 break

#             # Crop the frame to the selected bounding box
#             roi = frame[start_y:end_y, start_x:end_x]

#             # Perform YOLOv5 object detection on the cropped frame
#             results = self.yolo_model.predict(roi)

#             # Process the results (you can print or further process the detected objects here)

#             # Display the frame with bounding box and detected objects (you may need to adjust this based on your needs)
#             cv2.rectangle(frame, (start_x, start_y), (end_x, end_y), (0, 255, 0), 2)

#             cv2.imshow("YOLOv5 Detection", frame)

#             # Break the loop if 'q' key is pressed
#             if cv2.waitKey(1) & 0xFF == ord('q'):
#                 break

#         # Release video capture
#         video_capture.release()

#         # Destroy OpenCV windows
#         cv2.destroyAllWindows()

In [9]:
def main():
    root = tk.Tk()
    app = VideoUploaderApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()

In [9]:
!pip install yolov5

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Admin\\anaconda3\\Lib\\site-packages\\~il\\_imaging.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



     -------------------------------------- 953.4/953.4 kB 1.7 MB/s eta 0:00:00
     ---------------------------------------- 1.1/1.1 MB 1.4 MB/s eta 0:00:00
     -------------------------------------- 63.3/63.3 kB 424.3 kB/s eta 0:00:00
     ---------------------------------------- 88.3/88.3 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 807.9/807.9 kB 2.0 MB/s eta 0:00:00
     ------------------------------------ 190.6/190.6 kB 104.9 kB/s eta 0:00:00
     ------------------------------------ 645.2/645.2 kB 992.1 kB/s eta 0:00:00
     ---------------------------------------- 105.4/105.4 kB ? eta 0:00:00
     ------------------------------------ 311.7/311.7 kB 419.2 kB/s eta 0:00:00
     -------------------------------------- 62.7/62.7 kB 419.8 kB/s eta 0:00:00
     ------------------------------------ 166.4/166.4 kB 356.5 kB/s eta 0:00:00
     --------------------

  Using cached yolov5-7.0.13-py37.py38.py39.py310-none-any.whl (953 kB)
  Using cached huggingface_hub-0.19.4-py3-none-any.whl (311 kB)
  Using cached sahi-0.11.15-py3-none-any.whl (105 kB)
  Using cached torchvision-0.16.1-cp39-cp39-win_amd64.whl (1.1 MB)
  Using cached GitPython-3.1.40-py3-none-any.whl (190 kB)
  Using cached roboflow-1.1.9-py3-none-any.whl (63 kB)
  Using cached ultralytics-8.0.212-py3-none-any.whl (645 kB)
  Using cached setuptools-68.2.2-py3-none-any.whl (807 kB)
  Using cached fire-0.5.0.tar.gz (88 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached thop-0.1.1.post2209072238-py3-none-any.whl (15 kB)
  Using cached gitdb-4.0.11-py3-none-any.whl (62 kB)
  Using cached fsspec-2023.10.0-py3-none-any.whl (166 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl (54 kB)
  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)
  Using cached opencv_python_headless-4.8.0.74-cp37-abi3-win_am

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Admin\\anaconda3\\Lib\\site-packages\\cv2\\cv2.pyd'
Consider using the `--user` option or check the permissions.

